In [2]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client, progress, metrics
# wait for jobs to arrive, depending on the queue, this may take some time
import dask.array as da
import dask.bag as db
import numpy as np
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler
import os
import pyart


%matplotlib inline

In [ ]:
cluster = SLURMCluster(cores=36, project='acpc', walltime='0:20:00', 
                       job_cpu=36, memory='128GB', diagnostics_port=8787)
cluster.scale(6)         # Ask for ten workers
client = Client(cluster)  # Connect this local process to remote workers


In [ ]:
cluster

In [ ]:
client

In [ ]:
def my_fun(a):
    return a+1

In [ ]:
a = np.linspace(0,100, 100000)


In [ ]:
future = client.map(my_fun, a)


In [ ]:
progress(future)

In [ ]:
my_data = client.gather(future)


In [ ]:
def silly(fqdn):
    radar = pyart.io.read(fqdn)
    nt = radar.nsweeps
    del(radar)
    return nt

In [ ]:
idr = '/lcrc/group/earthscience/radar/xsapr_sgp/xsapri4case/'
files = os.listdir(idr)
asf = []
for ff in files:
    asf.append(idr+ff)

In [ ]:
len(asf)

In [ ]:
future = client.map(silly, asf)


In [ ]:
progress(future)

In [ ]:
my_data = client.gather(future)


In [ ]:
my_data